# transportation: Iceberg and Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Iceberg and Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using a transportation and logistics analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Iceberg?

Apache Iceberg is an open table format for huge analytic datasets that provides:

- **Schema evolution**: Add, drop, rename, update columns without rewriting data
- **Partition evolution**: Change partitioning without disrupting queries
- **Time travel**: Query historical data snapshots for auditing and rollback
- **ACID transactions**: Reliable concurrent read/write operations
- **Cross-engine compatibility**: Works with Spark, Flink, Presto, Hive, and more
- **Open ecosystem**: Apache 2.0 licensed, community-driven development

### Delta Universal Format with Iceberg

Delta Universal Format enables Iceberg compatibility while maintaining Delta's advanced features like liquid clustering. This combination provides:

- **Best of both worlds**: Delta's performance optimizations with Iceberg's openness
- **Multi-engine access**: Query the same data from different analytics engines
- **Future-proof architecture**: Standards-based approach for long-term data investments
- **Enhanced governance**: Rich metadata and catalog integration

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Fleet Management and Route Optimization

We'll analyze transportation fleet operations and logistics data. Our clustering strategy will optimize for:

- **Vehicle-specific queries**: Fast lookups by vehicle ID
- **Time-based analysis**: Efficient filtering by trip date and time
- **Route performance patterns**: Quick aggregation by route and operational metrics

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [1]:
# Create transportation catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS transportation")

spark.sql("CREATE SCHEMA IF NOT EXISTS transportation.analytics")

print("Transportation catalog and analytics schema created successfully!")

Transportation catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `fleet_trips_uf` table will store:

- **vehicle_id**: Unique vehicle identifier
- **trip_date**: Date and time of trip start
- **route_id**: Route identifier
- **distance**: Distance traveled (miles/km)
- **duration**: Trip duration (minutes)
- **fuel_consumed**: Fuel used (gallons/liters)
- **load_factor**: Capacity utilization (0-100)

### Clustering Strategy

We'll cluster by `vehicle_id` and `trip_date` because:

- **vehicle_id**: Vehicles generate multiple trips, grouping maintenance and performance data together
- **trip_date**: Time-based queries are essential for scheduling, fuel analysis, and operational reporting
- This combination optimizes for both vehicle monitoring and temporal fleet performance analysis

In [1]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
data_schema = StructType([
    StructField("vehicle_id", StringType(), True),
    StructField("trip_date", TimestampType(), True),
    StructField("route_id", StringType(), True),
    StructField("distance", DoubleType(), True),
    StructField("duration", DoubleType(), True),
    StructField("fuel_consumed", DoubleType(), True),
    StructField("load_factor", IntegerType(), True)
])
spark.sql("""

CREATE TABLE IF NOT EXISTS transportation.analytics.fleet_trips_uf (

    vehicle_id STRING,

    trip_date TIMESTAMP,

    route_id STRING,

    distance DECIMAL(8,2),

    duration DECIMAL(6,2),

    fuel_consumed DECIMAL(6,2),

    load_factor INT

)

USING DELTA

TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (vehicle_id, trip_date)

""")

print("Delta table with Iceberg compatibility and liquid clustering created successfully!")

print("Universal format enables Iceberg features while CLUSTER BY (columns) optimizes data layout.")

Delta table with Iceberg compatibility and liquid clustering created successfully!
Universal format enables Iceberg features while CLUSTER BY (columns) optimizes data layout.


## Step 3: Generate Transportation Sample Data

### Data Generation Strategy

We'll create realistic transportation fleet data including:

- **500 vehicles** with multiple trips over time
- **Route types**: Urban delivery, Long-haul, Local transport, Express delivery
- **Realistic operational patterns**: Peak hours, route variations, fuel efficiency differences
- **Fleet diversity**: Different vehicle types with varying capacities and fuel consumption

### Why This Data Pattern?

This data simulates real transportation scenarios where:

- Vehicle performance varies by route and time of day
- Fuel efficiency impacts operational costs
- Route optimization requires historical performance data
- Capacity utilization affects profitability
- Maintenance scheduling depends on usage patterns

In [1]:
# Generate sample transportation fleet data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define transportation data constants

ROUTE_TYPES = ['Urban Delivery', 'Long-haul', 'Local Transport', 'Express Delivery']

ROUTES = ['RT_NYC_MAN_001', 'RT_LAX_SFO_002', 'RT_CHI_DET_003', 'RT_HOU_DAL_004', 'RT_MIA_ORL_005']

# Base trip parameters by route type

TRIP_PARAMS = {

    'Urban Delivery': {'avg_distance': 45, 'avg_duration': 120, 'avg_fuel': 8.5, 'load_factor': 85},

    'Long-haul': {'avg_distance': 450, 'avg_duration': 480, 'avg_fuel': 65.0, 'load_factor': 92},

    'Local Transport': {'avg_distance': 120, 'avg_duration': 180, 'avg_fuel': 15.2, 'load_factor': 78},

    'Express Delivery': {'avg_distance': 80, 'avg_duration': 90, 'avg_fuel': 12.8, 'load_factor': 95}

}


# Generate fleet trip records

trip_data = []

base_date = datetime(2024, 1, 1)


# Create 500 vehicles with 20-60 trips each

for vehicle_num in range(1, 501):

    vehicle_id = f"VH{vehicle_num:04d}"
    
    # Each vehicle gets 20-60 trips over 12 months

    num_trips = random.randint(20, 60)
    
    for i in range(num_trips):

        # Spread trips over 12 months

        days_offset = random.randint(0, 365)

        trip_date = base_date + timedelta(days=days_offset)
        
        # Add realistic timing (more trips during business hours)

        hour_weights = [1, 1, 1, 1, 1, 3, 8, 10, 12, 10, 8, 6, 8, 9, 8, 7, 6, 5, 3, 2, 2, 1, 1, 1]

        hours_offset = random.choices(range(24), weights=hour_weights)[0]

        trip_date = trip_date.replace(hour=hours_offset, minute=random.randint(0, 59), second=0, microsecond=0)
        
        # Select route type

        route_type = random.choice(ROUTE_TYPES)

        params = TRIP_PARAMS[route_type]
        
        # Calculate trip metrics with variability

        distance_variation = random.uniform(0.7, 1.4)

        distance = round(params['avg_distance'] * distance_variation, 2)
        
        duration_variation = random.uniform(0.8, 1.6)

        duration = round(params['avg_duration'] * duration_variation, 2)
        
        fuel_variation = random.uniform(0.85, 1.25)

        fuel_consumed = round(params['avg_fuel'] * fuel_variation, 2)
        
        load_factor_variation = random.randint(-10, 8)

        load_factor = max(0, min(100, params['load_factor'] + load_factor_variation))
        
        # Select specific route

        route_id = random.choice(ROUTES)
        
        trip_data.append({

            "vehicle_id": vehicle_id,

            "trip_date": trip_date,

            "route_id": route_id,

            "distance": distance,

            "duration": duration,

            "fuel_consumed": fuel_consumed,

            "load_factor": load_factor

        })



print(f"Generated {len(trip_data)} fleet trip records")

print("Sample record:", trip_data[0])

Generated 20312 fleet trip records
Sample record: {'vehicle_id': 'VH0001', 'trip_date': datetime.datetime(2024, 10, 10, 8, 46), 'route_id': 'RT_MIA_ORL_005', 'distance': 85.38, 'duration': 147.54, 'fuel_consumed': 14.27, 'load_factor': 84}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [1]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_trips = spark.createDataFrame(trip_data, schema=data_schema)


# Display schema and sample data

print("DataFrame Schema:")

df_trips.printSchema()



print("\nSample Data:")

df_trips.show(5)


# Insert data into Delta table with liquid clustering

# The TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (vehicle_id, trip_date) will automatically optimize the data layout

df_trips.write.mode("overwrite").insertInto("transportation.analytics.fleet_trips_uf")


print(f"\nSuccessfully inserted {df_trips.count()} records into transportation.analytics.fleet_trips_uf")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- vehicle_id: string (nullable = true)
 |-- trip_date: timestamp (nullable = true)
 |-- route_id: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- fuel_consumed: double (nullable = true)
 |-- load_factor: integer (nullable = true)


Sample Data:
+----------+-------------------+--------------+--------+--------+-------------+-----------+
|vehicle_id|          trip_date|      route_id|distance|duration|fuel_consumed|load_factor|
+----------+-------------------+--------------+--------+--------+-------------+-----------+
|    VH0001|2024-10-10 08:46:00|RT_MIA_ORL_005|   85.38|  147.54|        14.27|         84|
|    VH0001|2024-02-06 14:32:00|RT_CHI_DET_003|  152.57|  286.72|        15.04|         71|
|    VH0001|2024-03-30 12:31:00|RT_MIA_ORL_005|   87.53|  131.81|        14.84|         98|
|    VH0001|2024-12-19 08:55:00|RT_MIA_ORL_005|   60.46|   72.99|        13.81|         86|
|    VH0001|2024-08-24 


Successfully inserted 20312 records into transportation.analytics.fleet_trips_uf
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Vehicle trip history** (clustered by vehicle_id)
2. **Time-based fleet analysis** (clustered by trip_date)
3. **Combined vehicle + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [1]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: Vehicle trip history - benefits from vehicle_id clustering

print("=== Query 1: Vehicle Trip History ===")

vehicle_history = spark.sql("""

SELECT vehicle_id, trip_date, route_id, distance, fuel_consumed, load_factor

FROM transportation.analytics.fleet_trips_uf

WHERE vehicle_id = 'VH0001'

ORDER BY trip_date DESC

""")



vehicle_history.show()

print(f"Records found: {vehicle_history.count()}")



# Query 2: Time-based fuel efficiency analysis - benefits from trip_date clustering

print("\n=== Query 2: Recent Fuel Efficiency Issues ===")

fuel_efficiency = spark.sql("""

SELECT trip_date, vehicle_id, route_id, distance, fuel_consumed,

       ROUND(distance / fuel_consumed, 2) as mpg

FROM transportation.analytics.fleet_trips_uf

WHERE trip_date >= '2024-06-01' AND (distance / fuel_consumed) < 15

ORDER BY mpg ASC, trip_date DESC

""")



fuel_efficiency.show()

print(f"Fuel efficiency issues found: {fuel_efficiency.count()}")



# Query 3: Combined vehicle + time query - optimal for our clustering strategy

print("\n=== Query 3: Vehicle Performance Trends ===")

performance_trends = spark.sql("""

SELECT vehicle_id, trip_date, route_id, distance, duration, load_factor

FROM transportation.analytics.fleet_trips_uf

WHERE vehicle_id LIKE 'VH000%' AND trip_date >= '2024-04-01'

ORDER BY vehicle_id, trip_date

""")



performance_trends.show()

print(f"Performance trend records found: {performance_trends.count()}")

=== Query 1: Vehicle Trip History ===


+----------+-------------------+--------------+--------+-------------+-----------+
|vehicle_id|          trip_date|      route_id|distance|fuel_consumed|load_factor|
+----------+-------------------+--------------+--------+-------------+-----------+
|    VH0001|2024-12-29 15:51:00|RT_CHI_DET_003|  111.35|        17.40|         68|
|    VH0001|2024-12-19 08:55:00|RT_MIA_ORL_005|   60.46|        13.81|         86|
|    VH0001|2024-11-13 10:55:00|RT_CHI_DET_003|   72.79|        13.53|         90|
|    VH0001|2024-10-25 04:51:00|RT_LAX_SFO_002|   85.48|        11.69|         96|
|    VH0001|2024-10-23 12:26:00|RT_HOU_DAL_004|  471.28|        68.73|         85|
|    VH0001|2024-10-22 06:37:00|RT_HOU_DAL_004|  163.80|        18.49|         82|
|    VH0001|2024-10-13 14:52:00|RT_CHI_DET_003|   37.25|         9.50|         85|
|    VH0001|2024-10-10 08:46:00|RT_MIA_ORL_005|   85.38|        14.27|         84|
|    VH0001|2024-10-01 07:10:00|RT_MIA_ORL_005|  162.44|        14.26|         72|
|   

Records found: 33

=== Query 2: Recent Fuel Efficiency Issues ===


+-------------------+----------+--------------+--------+-------------+----+
|          trip_date|vehicle_id|      route_id|distance|fuel_consumed| mpg|
+-------------------+----------+--------------+--------+-------------+----+
|2024-10-25 15:02:00|    VH0251|RT_HOU_DAL_004|   31.67|        10.58|2.99|
|2024-09-15 20:09:00|    VH0009|RT_HOU_DAL_004|   31.58|        10.57|2.99|
|2024-06-11 14:34:00|    VH0495|RT_HOU_DAL_004|   31.52|        10.50|3.00|
|2024-08-29 17:51:00|    VH0362|RT_MIA_ORL_005|   31.84|        10.57|3.01|
|2024-11-15 11:06:00|    VH0340|RT_CHI_DET_003|   31.66|        10.39|3.05|
|2024-07-07 06:57:00|    VH0112|RT_NYC_MAN_001|   31.69|        10.34|3.06|
|2024-12-10 13:14:00|    VH0078|RT_LAX_SFO_002|   31.99|        10.41|3.07|
|2024-11-12 15:04:00|    VH0377|RT_CHI_DET_003|   32.21|        10.47|3.08|
|2024-12-28 07:10:00|    VH0157|RT_CHI_DET_003|   32.33|        10.47|3.09|
|2024-12-05 13:27:00|    VH0195|RT_MIA_ORL_005|   32.54|        10.50|3.10|
|2024-11-16 

Fuel efficiency issues found: 11883

=== Query 3: Vehicle Performance Trends ===


+----------+-------------------+--------------+--------+--------+-----------+
|vehicle_id|          trip_date|      route_id|distance|duration|load_factor|
+----------+-------------------+--------------+--------+--------+-----------+
|    VH0001|2024-04-15 02:39:00|RT_CHI_DET_003|   41.83|  123.93|         76|
|    VH0001|2024-04-20 09:26:00|RT_NYC_MAN_001|   51.83|  125.91|         91|
|    VH0001|2024-04-20 12:23:00|RT_HOU_DAL_004|   59.45|  164.47|         90|
|    VH0001|2024-04-26 12:41:00|RT_MIA_ORL_005|  355.92|  681.87|         88|
|    VH0001|2024-06-02 09:47:00|RT_CHI_DET_003|  116.68|  200.46|         76|
|    VH0001|2024-06-06 11:27:00|RT_NYC_MAN_001|   75.87|   80.11|         88|
|    VH0001|2024-06-16 08:19:00|RT_MIA_ORL_005|  131.28|  278.89|         73|
|    VH0001|2024-06-25 17:42:00|RT_CHI_DET_003|  121.93|  282.40|         74|
|    VH0001|2024-07-13 14:12:00|RT_NYC_MAN_001|  394.12|  510.99|         82|
|    VH0001|2024-07-15 15:17:00|RT_MIA_ORL_005|  421.39|  485.49

Performance trend records found: 250


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the transportation insights possible with this optimized structure.

### Key Analytics

- **Vehicle utilization** and performance metrics
- **Route efficiency** and fuel consumption analysis
- **Fleet capacity utilization** and load factors
- **Operational cost trends** and optimization opportunities

In [1]:
# Analyze clustering effectiveness and transportation insights


# Vehicle performance analysis

print("=== Vehicle Performance Analysis ===")

vehicle_performance = spark.sql("""

SELECT vehicle_id, COUNT(*) as total_trips,

       ROUND(SUM(distance), 2) as total_distance,

       ROUND(SUM(fuel_consumed), 2) as total_fuel,

       ROUND(AVG(distance / fuel_consumed), 2) as avg_mpg,

       ROUND(AVG(load_factor), 2) as avg_load_factor,

       ROUND(SUM(distance), 0) as total_miles

FROM transportation.analytics.fleet_trips_uf

GROUP BY vehicle_id

ORDER BY total_miles DESC

""")



vehicle_performance.show()


# Route efficiency analysis

print("\n=== Route Efficiency Analysis ===")

route_efficiency = spark.sql("""

SELECT route_id, COUNT(*) as total_trips,

       ROUND(AVG(distance), 2) as avg_distance,

       ROUND(AVG(duration), 2) as avg_duration,

       ROUND(AVG(distance / duration * 60), 2) as avg_speed,

       ROUND(AVG(load_factor), 2) as avg_load_factor

FROM transportation.analytics.fleet_trips_uf

GROUP BY route_id

ORDER BY total_trips DESC

""")



route_efficiency.show()


# Fleet fuel consumption analysis

print("\n=== Fleet Fuel Consumption Analysis ===")

fuel_analysis = spark.sql("""

SELECT 

    CASE 

        WHEN distance / fuel_consumed >= 25 THEN 'Excellent (25+ MPG)'

        WHEN distance / fuel_consumed >= 20 THEN 'Good (20-24 MPG)'

        WHEN distance / fuel_consumed >= 15 THEN 'Average (15-19 MPG)'

        WHEN distance / fuel_consumed >= 10 THEN 'Poor (10-14 MPG)'

        ELSE 'Very Poor (<10 MPG)'

    END as fuel_efficiency_category,

    COUNT(*) as trip_count,

    ROUND(AVG(distance / fuel_consumed), 2) as avg_mpg,

    ROUND(SUM(fuel_consumed), 2) as total_fuel_used

FROM transportation.analytics.fleet_trips_uf

GROUP BY 

    CASE 

        WHEN distance / fuel_consumed >= 25 THEN 'Excellent (25+ MPG)'

        WHEN distance / fuel_consumed >= 20 THEN 'Good (20-24 MPG)'

        WHEN distance / fuel_consumed >= 15 THEN 'Average (15-19 MPG)'

        WHEN distance / fuel_consumed >= 10 THEN 'Poor (10-14 MPG)'

        ELSE 'Very Poor (<10 MPG)'

    END

ORDER BY avg_mpg DESC

""")



fuel_analysis.show()


# Monthly operational trends

print("\n=== Monthly Operational Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(trip_date, 'yyyy-MM') as month,

       COUNT(*) as total_trips,

       ROUND(SUM(distance), 2) as monthly_distance,

       ROUND(SUM(fuel_consumed), 2) as monthly_fuel,

       ROUND(AVG(load_factor), 2) as avg_load_factor,

       COUNT(DISTINCT vehicle_id) as active_vehicles

FROM transportation.analytics.fleet_trips_uf

GROUP BY DATE_FORMAT(trip_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== Vehicle Performance Analysis ===


+----------+-----------+--------------+----------+-------+---------------+-----------+
|vehicle_id|total_trips|total_distance|total_fuel|avg_mpg|avg_load_factor|total_miles|
+----------+-----------+--------------+----------+-------+---------------+-----------+
|    VH0219|         56|      14414.62|   1996.12|   7.08|          88.43|      14415|
|    VH0247|         58|      14326.24|   2070.42|   6.74|          88.19|      14326|
|    VH0155|         60|      13596.17|   1873.72|   6.96|           86.2|      13596|
|    VH0181|         57|      13083.02|   1808.55|   6.75|          86.74|      13083|
|    VH0076|         55|      12917.05|   1806.19|   6.76|          87.67|      12917|
|    VH0450|         58|      12698.26|   1848.91|   6.32|          87.47|      12698|
|    VH0131|         58|      12606.90|   1741.35|   6.88|          86.19|      12607|
|    VH0105|         48|      12235.67|   1784.09|   6.76|           87.9|      12236|
|    VH0311|         60|      12221.40|   1

+--------------+-----------+------------+------------+---------+---------------+
|      route_id|total_trips|avg_distance|avg_duration|avg_speed|avg_load_factor|
+--------------+-----------+------------+------------+---------+---------------+
|RT_LAX_SFO_002|       4113|      179.23|      258.82|    38.84|          86.45|
|RT_CHI_DET_003|       4089|      185.82|      266.20|    39.07|          86.53|
|RT_MIA_ORL_005|       4068|      187.58|      265.81|    39.50|          86.49|
|RT_NYC_MAN_001|       4043|      182.72|      260.98|    39.05|          86.59|
|RT_HOU_DAL_004|       3999|      180.90|      258.65|    39.16|           86.6|
+--------------+-----------+------------+------------+---------+---------------+


=== Fleet Fuel Consumption Analysis ===


+------------------------+----------+-------+---------------+
|fuel_efficiency_category|trip_count|avg_mpg|total_fuel_used|
+------------------------+----------+-------+---------------+
|        Poor (10-14 MPG)|       918|  10.86|       20974.34|
|     Very Poor (<10 MPG)|     19394|   6.45|      523896.77|
+------------------------+----------+-------+---------------+


=== Monthly Operational Trends ===


+-------+-----------+----------------+------------+---------------+---------------+
|  month|total_trips|monthly_distance|monthly_fuel|avg_load_factor|active_vehicles|
+-------+-----------+----------------+------------+---------------+---------------+
|2024-01|       1676|       310018.60|    45557.94|          86.51|            477|
|2024-02|       1655|       297821.65|    43499.51|          86.69|            472|
|2024-03|       1712|       313838.77|    45533.54|          86.75|            484|
|2024-04|       1644|       291398.21|    42949.60|          86.44|            477|
|2024-05|       1742|       320434.99|    47059.70|          86.44|            479|
|2024-06|       1662|       289073.59|    42807.33|          86.58|            478|
|2024-07|       1720|       315320.33|    45324.33|          86.37|            468|
|2024-08|       1775|       318035.62|    46701.85|          86.35|            477|
|2024-09|       1652|       312432.63|    45628.41|          86.94|         

## Key Takeaways: Iceberg and Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (vehicle_id, trip_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (vehicle_id, trip_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Transportation analytics where fleet monitoring and route optimization are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for transportation data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles transportation-scale data volumes effortlessly

### Best Practices for Iceberg and Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger transportation datasets
- Integrate with real GPS tracking and IoT sensor data

This notebook demonstrates how Oracle AI Data Platform combines Delta's advanced liquid clustering with Iceberg's open, future-proof architecture to deliver enterprise-grade analytics that are both high-performance and standards-compliant.